In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [5]:
#id first_training
#caption Results from the first training
# CLICK ME
from fastai.vision.all import *
path = untar_data(URLs.PETS)/'images'

def is_cat(x): return x[0].isupper()
dls = ImageDataLoaders.from_name_func(
    path, get_image_files(path), valid_pct=0.2, seed=42,
    label_func=is_cat, item_tfms=Resize(224))

learn = cnn_learner(dls, resnet34, metrics=error_rate)
learn.fine_tune(1)

/opt/conda/lib/python3.10/site-packages/fastai/vision/learner.py:288: UserWarning: `cnn_learner` has been renamed to `vision_learner` -- please update your code
  warn("`cnn_learner` has been renamed to `vision_learner` -- please update your code")
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593

epoch,train_loss,valid_loss,error_rate,time
0,0.153300,0.031083,0.009472,17:37


epoch,train_loss,valid_loss,error_rate,time
0,0.074996,0.029109,0.010825,25:51


In [9]:
import ipywidgets as widgets
uploader =  widgets.FileUpload()
uploader

FileUpload(value={}, description='Upload')

In [14]:
img = PILImage.create(uploader.data[0])
is_cat,_,probs = learn.predict(img)
print(f"Is this a cat?: {is_cat}.")
print(f"Probability it's a cat: {probs[1].item():.6f}")

Is this a cat?: True.
Probability it's a cat: 1.000000


#### 1.6.7 이미지 인식 모델의 작동 방식

In [2]:
from fastai.vision.all import *
path = untar_data(URLs.PETS)/'images'
# 두번째 줄은 fast.ai의 데이터셋 컬렉션에서 데이터 셋을 서버에 다운로드하고, 압축을 해제한 다음
# (둘 다 한 적이 없다면)
# 추출된 위치를 Path 객체로 반환한다.

In [4]:
def is_cat(x): return x[0].isupper()
# 데이터셋에 포함된 파일명의 규칙에 따라 고양이 레이블을 지정하는 is_cat 함수를 정의함

In [ ]:
dls = ImageDataLoaders.from_name_func(
    path, get_image_files(path), valid_pct=0.2, seed=42,
    label_func=is_cat, item_tfms=Resize(224))

# is_cat 함수를 사용해 fastai에 데이터셋의 종류와 구조화 방법을 알려줌.
# fastai는 다양한 딥러닝 데이터셋과 문제에 사용할 수 있는 여러 클래스를 제공. (여기서는 ImageDataLoaders)
# fastai는 데이터셋에서 레이블을 가져오는 방법도 알아야 함. (label_func = is_cat)
# 마지막 부분은 필요한 Transform(변형) 목록을 지정.
# Transform은 학습 중 자동으로 적용되는 코드를 포함함.
# fastai는 미리 정의된 여러가지 Transform을 제공. 변형의 두 종류 -> item_tfms & batch_tfms
# item_tfms: 개별 데이터에 적용할 변형을 지정
# batch_tfms: 데이터 배치에 GPU가 빠르게 한 번에 처리할 변형을 지정하는 데 사용.

# 224 픽셀을 지정한 이유: 영상 처리 분야의 역사적 이유. 흔히 찾아볼 수 있는 표준적인 크기이기 때문.
# 크기를 늘리면 더 나은 결과를 얻을 수 있지만, 속도와 메모리 소비량을 희생해야 한다.

# !중요! vali_pct=0.2 -> 데이터의 20%를 모델 학습에 사용하지 말고 따로 빼놓으라고 지시하는 역할.
# 나머지 20% 데이터를 검증용 데이터셋 validation set, 나머지 80%를 학습용 데이터셋 training set
# 검증용 -> 모델의 정확도를 측정하는 데 사춍. 보통 임의로 선택헤서 구성.

# seed=42 -> 코드를 실행할 때마다 동일한 임의성으로 검증용 데이터셋을 구성하도록 시드값을 고정한 부분.(랜덤시드)

In [ ]:
learn = cnn_learner(dls, resnet34, metrics=error_rate)

# 합성곱 신경망 Convolutional neural network (CNN)을 생성
# 이때 학습에 사용할 데이터셋과 모델의 아키텍처(모델의 종류) 및 평가지표 metric을 지정.
# 왜 CNN 사용? 영상 처리 모델을 만드는 최신 접근법이기 때문.

# resnet34 -> 34는 ResNet이라는 구조의 모델에 있는 계층의 수를 나타냄.
# 더 많은 계층 구조가 있는 모델은 학습에 더 오랜 시간이 걸리며, 더 과적합되는 경향이 있음.

# 평가지표란 검증용 데이터셋을 사용해 모델이 내놓은 예측의 품질을 측정하는 함수.
# 보통 매 에포크(학습 횟수)의 마지막에 출력. 여기서는 fastai error_rate(오차율) 함수를 사용.
# 정확도: accuracy = 1.0 - error_rate 로 단순 계산.

cnn_learner 함수에는 pretrained 라는 인자가 있음. 기본값인 True이면, 전문가가 약 천 개의 범주로 구성된 130만 장의 사진(유명한 데이터셋)의 학습으로 얻은 가중치를 모델의 가중치로 설정함.
다른 데이터셋에서 이미 학습된 가중치가 있는 모델을 **사전 학습된 모델 pretrained model** 이라고 함.
학습용 데이터가 전혀 준비되지 않은 상황에서도 이미 매우 유능한 모델이기 때문에 가능하면 사전 학습된 모델을 사용하기를 권장.

사전 학습된 모델을 사용하는 방법은 데이터, 시간, 비용을 적게 들이면서 빠르고 정확한 모델을 학습시키는 가장 중요한 기법임. 

원래 학습된 작업과는 다른 작업에 사전 학습된 모델을 사용하는 일을 **전이 학습 transfer learning** 이라고 함.

In [ ]:
learn.fine_tune(1)

# 모델을 적합 fit 시키기를 지시.
# 사전 학습된 모델로 시작했기 때문에 이미 학습된 모든 가능성을 유지하면서 새로운 데이터셋에 모델을 적응시키는 과정
# -> 미세조정 fine tuning

In [ ]:
from fastai.text.all import *

dls = TextDataLoaders.from_folder(untar_data(URLs.IMDB), valid='test')
learn = text_classifier_learner(dls, AWD_LSTM, drop_mult=0.5, metrics=accuracy)
learn.fine_tune(4, 1e-2)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

epoch,train_loss,valid_loss,accuracy,time


In [ ]:
learn.predict("I really liked that movie!")

In [ ]:
# 사람들의 사회경제적 배경을 토대로 고소득자인지를 예측하는 모델을 학습시키는 코드.
from fastai.tabular.all import *
path = untar_data(URLs.ADULT_SAMPLE)

dls = TabularDataLoaders.from_csv(path/'adult.csv', path=path, y_names="salary",
                                 cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race'],
                                 cont_names = ['age', 'fnlwgt', 'education-num'],
                                 procs = [Categorify, FillMissing, Normalize])
learn = tabular_learner(dls, metrics=accuracy)

In [ ]:
learn.fit_one_cycle(3)